In [ ]:
from transformers import pipeline
import pandas as pd
import torch

In [ ]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=0 if torch.cuda.is_available() else -1
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv("ai_center_comments.csv")

In [ ]:
def get_sentiment(text):
    try:
        result = sentiment_model(text[:512])[0]  # truncate long comments
        return result["label"], result["score"]
    except:
        return None, None

df[["sentiment", "confidence"]] = df["text"].apply(
    lambda x: pd.Series(get_sentiment(str(x)))
)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

df["sentiment"] = df["sentiment"].map(label_map)

In [ ]:
df

,text,username,sentiment,confidence
0,Big news!!,cmetz_wynn,positive,0.934147
1,NaN,poiyoc,neutral,0.528189
2,WHO ASKED FOR THIS 😭😭💔💔💔,loise07,negative,0.920092
3,*$55 million child pornography machine,gavin.creaser,negative,0.852920
4,"Gross. Disgusting. Why do you need money, afte...",mcknight.theatre,negative,0.977026
...,...,...,...,...
551,New wrestling room.,sparacio,neutral,0.854198
552,So we can’t shower anymore,nessasedano,negative,0.714538
553,this is so lame. disappointed bing,britt.lemasters,negative,0.971832
554,or we could spend this on food that is cooked ...,masharmellow,neutral,0.638501


In [ ]:
neu = df[df["sentiment"] == "neutral"]["text"]

In [ ]:
neu = neu.dropna().reset_index()['text']

In [ ]:
neu.to_csv("neutral.csv")

In [ ]:
pos = df[df["sentiment"] == "positive"]["text"]

In [ ]:
pos = pos.dropna().reset_index()['text']

In [ ]:
pos.to_csv("positive.csv")

In [ ]:
neg = df[df["sentiment"] == "negative"]["text"]

In [ ]:
neg = neg.dropna().reset_index()['text']

In [ ]:
neg.to_csv("negative.csv")